In [ ]:
# Author: Chelsea Ekstrand <chelsea.ekstrand@uleth.ca>

In [1]:
# Import functions

import numpy as np
import re
import nibabel as nib
from glob import glob
from os import path

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(l, key=alphanum_key)

In [2]:
# Set parameters

fmri_path = '/media/ekstrand/BackupPlus2/NaturalisticDataset_v2/'# path to subject directories in BIDS format
cond1_path = path.join(fmri_path, 'face_timings_for_FIR') # path to onset and duration text files for condition one
cond2_path = path.join(fmri_path, 'noface_timings_for_FIR')# path to onset and duration text files for condition one
cond1name = 'face'
cond2name = 'noface'
task = '500daysofsummer'# BIDS task name
suffix = 'bold_no_blur_no_censor_ica'# end of proprocessed filename after task
saveDir_cond1 = path.join(fmri_path,'sub_face_averages') # path to save averaged fMRI files for condition 1
saveDir_cond2 = path.join(fmri_path,'sub_noface_averages') # path to save averaged fMRI files for condition 1
time = 11 # number of timepoints to model
lag = 3 # how many seconds after the event to model to account for the slow BOLD function
TR = 1 # fMRI repetition time
saveSubs = 'n' # save individual subject averages? 'y'es or 'n'o

In [ ]:
def get_condition_timing_data(fmri_data,beh_data,fill,tr,volumes):
    ind = 0
    for c in beh_data:
        # calculate the onset volume by dividing the onset time by the TR, rounding down to nearest int
        vol = int((c[0]/tr) + lag) 
        print(vol)
        fill[:,:,:,:,ind] = fmri_data[:,:,:,vol:vol+volumes]
        ind = ind + 1

    #calculate average of all the condition 1 instances
    ave = np.mean(fill,axis=4)

    # reshape data to time, x,y,z format needed for clustering
    ave = np.transpose(ave,(3,0,1,2))
    return(ave)

def restructure_data(subs,c1_path,c2_path,sSubs):
    count = 0
    for s in subs:
        sub_num = s.split('/')[-1].split('_')[0]
        print(sub_num)
        cond1file = glob(path.join(c1_path,sub_num + '_*.txt'))
        cond2file = glob(path.join(c2_path,sub_num + '_*.txt'))

        # load functional data
        load_data = nib.load(s)
        data = load_data.get_fdata()
        x,y,z,t = np.shape(data)
        print(x,y,z,t)

        # load event data
        cond1_data = np.loadtxt(cond1file[0],delimiter=' ')
        cond2_data = np.loadtxt(cond2file[0],delimiter=' ')

        cond_fill = np.zeros((x,y,z,time,len(cond1_data)))

        cond1_ave = get_condition_timing_data(data,cond1_data,cond_fill,TR,volumes)
        cond2_ave = get_condition_timing_data(data,cond2_data,cond_fill,TR,volumes)

        master_fill_cond1[count,:,:,:,:] = cond1_ave
        master_fill_cond2[count,:,:,:,:] = cond2_ave

        if sSubs == 'y':
            # save condition 1 average file for each subject
            saveFileCond1 = path.join(saveDir_cond1,sub_num + '_task-' + task + '_cond-' + cond1name + '_TRtimepoints-' + str(t) + '_average.nii.gz')
            saveCond1 = nib.Nifti1Image(cond1_ave,load_data.affine)
            nib.save(saveCond1,saveFileCond1)

            # save condition 1 average file for each subject
            saveFileCond2 = path.join(saveDir_cond2,sub_num + '_task-' + task + '_cond-' + cond2name + '_TRtimepoints-' + str(t) + '_average.nii.gz')
            saveCond2 = nib.Nifti1Image(cond2_ave,load_data.affine)
            nib.save(saveCond2,saveFileCond2)   

        count = count+1
    return(master_fill_cond1,master_fill_cond2)


In [ ]:
# restructure functional data for spatiotemporal clustering



subjects = glob(path.join(fmri_path,'sub-*/func/sub-*_task-' + task + '_' + suffix + '.nii.gz'))
subjects = natural_sort(subjects)

dim_data = nib.load(subjects[0])
get_dim_data = dim_data.get_fdata()
dim_x,dim_y,dim_z,dim_t = np.shape(get_dim_data)
volumes = int(time/TR)

#create new matrices to fill for saving
master_fill_cond1 = np.empty((len(subjects),volumes,dim_x,dim_y,dim_z))
master_fill_cond2 = np.empty((len(subjects),volumes,dim_x,dim_y,dim_z))

print(np.shape(master_fill_cond1),np.shape(master_fill_cond2))

if time % TR != 0:
    print('ERROR: Desired timepoints not divisible by TR!')
elif path.exists(path.join(saveDir_cond2,str(len(subjects)) + 'sub_average_task-' + task + '_cond-' + cond2name + '_TRtimepoints-' + str(volumes) + '_lag-' + str(lag) + '.nii.gz')) == False:
    big_fill_cond1, big_fill_cond2 = restructure_data(subjects,cond1_path,cond2_path,saveSubs)
    
    saveCond1masterFile = path.join(saveDir_cond1,str(len(subjects)) + 'sub_average_task-' + task + '_cond-' + cond1name + '_TRtimepoints-' + str(volumes) + '_lag-' + str(lag) + '.nii.gz')
    saveCond2masterFile = path.join(saveDir_cond2,str(len(subjects)) + 'sub_average_task-' + task + '_cond-' + cond2name + '_TRtimepoints-' + str(volumes) + '_lag-' + str(lag) + '.nii.gz')
    
    saveCond1master = nib.Nifti1Image(big_fill_cond1,dim_data.affine)
    nib.save(saveCond1master,saveCond1masterFile)
    saveCond2master = nib.Nifti1Image(big_fill_cond2,dim_data.affine)
    nib.save(saveCond2master,saveCond2masterFile)
else:
    print('File already exists!')